In [1]:
# -*- coding: utf-8 -*-
""" 
Usage:
    THEANO_FLAGS="device=gpu0" python exptBasestationXIAN.py
"""
from __future__ import print_function
import sys
sys.path.insert(0, '..')
import os
import pickle
import numpy as np
import math
import h5py

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from deepst.models.STResNet import stresnet
from deepst.config import Config
import deepst.metrics as metrics
from deepst.datasets import ShenyangRegular
np.random.seed(1337)  # for reproducibility

# parameters
# data path, you may set your own data path with the global envirmental
# variable DATAPATH
DATAPATH = Config().DATAPATH
nb_epoch = 500  # number of epoch at training stage
nb_epoch_cont = 100  # number of epoch at training (cont) stage
batch_size = 32  # batch size
T = 48  # number of time intervals in one day
nbfilter=64
lr_arr = [0.0002, 0.0005, 0.002, 0.005, 0.02, 0.05, 0.2, 0.5]  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence
nb_residual_unit = 4   # number of residual units

nb_flow = 2  # there are two types of flows: new-flow and end-flow
# divide data into two subsets: Train & Test, of which the test set is the
# last 10 days
days_test = 4
len_test = T * days_test
map_height, map_width = 10, 10  # grid size
# For NYC Bike data, there are 81 available grid-based areas, each of
# which includes at least ONE bike station. Therefore, we modify the final
# RMSE by multiplying the following factor (i.e., factor).
nb_area = 10 * 10
m_factor = math.sqrt(1. * map_height * map_width / nb_area)
print('factor: ', m_factor)
path_result = 'RET'
path_model = 'MODEL'

if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)


/home/csi/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


factor:  1.0


In [2]:
def build_model(external_dim, lr, nbfilter=64):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None

    model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit, nbfilter=nbfilter)
    adam = Adam(lr=lr)
    model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse])
    model.summary()
    # from keras.utils.visualize_util import plot
    # plot(model, to_file='model.png', show_shapes=True)
    return model


def main(lr):
    # load data
    print("loading data...")
    X_train, Y_train, X_test, Y_test, mmn, external_dim, timestamp_train, timestamp_test = ShenyangRegular.load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        preprocess_name='preprocessing.pkl', meta_data=False)

    print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])

    print('=' * 10)
    print("compiling model...")
    print(
        "**at the first time, it takes a few minites to compile if you use [Theano] as the backend**")
    print("before build_model")
    model = build_model(external_dim, lr, nbfilter=nbfilter)
    print("after build_model")
    hyperparams_name = 'c{}.p{}.t{}.resunit{}.lr{}'.format(
        len_closeness, len_period, len_trend, nb_residual_unit, lr)
    fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

    early_stopping = EarlyStopping(monitor='val_rmse', patience=5, mode='min')
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

    print('=' * 10)
    print("training model...")
    history = model.fit(X_train, Y_train,
                        nb_epoch=nb_epoch,
                        batch_size=batch_size,
                        validation_split=0.143,
                        callbacks=[early_stopping, model_checkpoint],
                        verbose=2)
    model.save_weights(os.path.join(
        'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))

    print('=' * 10)
    print('evaluating using the model that has the best loss on the valid set')

    model.load_weights(fname_param)
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) * m_factor))

    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) * m_factor))

    print('=' * 10)
    print("training model (cont)...")
    fname_param = os.path.join(
        'MODEL', '{}.cont.best.h5'.format(hyperparams_name))
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='rmse', verbose=0, save_best_only=True, mode='min')
    history = model.fit(X_train, Y_train, nb_epoch=nb_epoch_cont, verbose=1, batch_size=batch_size, callbacks=[
                        model_checkpoint], validation_data=(X_test, Y_test))
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.cont.history.pkl'.format(hyperparams_name)), 'wb'))
    model.save_weights(os.path.join(
        'MODEL', '{}_cont.h5'.format(hyperparams_name)), overwrite=True)

    print('=' * 10)
    print('evaluating using the final model')
     
    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[
                           0] // 48, verbose=0)
    print('Train score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) * m_factor))

    score2 = model.predict(X_test, batch_size=Y_test.shape[0], verbose=0)
    file_temp = h5py.File("prediction", "w")
    file_temp.create_dataset("prediction", data=score2)
    file_temp.create_dataset("real", data=Y_test)
    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) * m_factor))

if __name__ == '__main__':
    for lr_value in lr_arr:
        main(lr_value)
    
    

loading data...
incomplete days:  []
train_data shape:  (1152, 2, 10, 10)
min: 0.0 max: 1344.0
XC shape:  (1008, 6, 10, 10) XP shape:  (1008, 2, 10, 10) XT shape:  (1008, 2, 10, 10) Y shape: (1008, 2, 10, 10)
XC shape:  (1008, 6, 10, 10) XP shape:  (1008, 2, 10, 10) XT shape:  (1008, 2, 10, 10) Y shape: (1008, 2, 10, 10)
train shape: (816, 6, 10, 10) (816, 2, 10, 10) test shape:  (192, 6, 10, 10) (192, 2, 10, 10)
(816, 6, 10, 10)
(816, 2, 10, 10)
(816, 2, 10, 10)

(192, 6, 10, 10)
(192, 2, 10, 10)
(192, 2, 10, 10)


 days (test):  [b'20150113', b'20150114', b'20150115', b'20150116']
compiling model...
**at the first time, it takes a few minites to compile if you use [Theano] as the backend**
before build_model
external_dim: None
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 6, 10, 10)     0   

Train on 699 samples, validate on 117 samples
Epoch 1/500
5s - loss: 0.0128 - rmse: 0.0926 - val_loss: 0.0023 - val_rmse: 0.0470
Epoch 2/500
1s - loss: 0.0021 - rmse: 0.0454 - val_loss: 0.0013 - val_rmse: 0.0360
Epoch 3/500
1s - loss: 0.0013 - rmse: 0.0363 - val_loss: 0.0012 - val_rmse: 0.0343
Epoch 4/500
1s - loss: 9.7868e-04 - rmse: 0.0311 - val_loss: 7.5414e-04 - val_rmse: 0.0272
Epoch 5/500
2s - loss: 8.1862e-04 - rmse: 0.0284 - val_loss: 6.8240e-04 - val_rmse: 0.0259
Epoch 6/500
1s - loss: 8.1509e-04 - rmse: 0.0282 - val_loss: 6.9822e-04 - val_rmse: 0.0263
Epoch 7/500
2s - loss: 6.3244e-04 - rmse: 0.0249 - val_loss: 7.9106e-04 - val_rmse: 0.0280
Epoch 8/500
2s - loss: 6.5012e-04 - rmse: 0.0252 - val_loss: 6.2590e-04 - val_rmse: 0.0249
Epoch 9/500
1s - loss: 6.7262e-04 - rmse: 0.0257 - val_loss: 6.3282e-04 - val_rmse: 0.0250
Epoch 10/500
1s - loss: 5.5583e-04 - rmse: 0.0234 - val_loss: 6.6512e-04 - val_rmse: 0.0257
Epoch 11/500
2s - loss: 5.4708e-04 - rmse: 0.0232 - val_loss: 5.635

816/816 [==============================] - 2s - loss: 3.2721e-04 - rmse: 0.0178 - val_loss: 7.6035e-04 - val_rmse: 0.0271
Epoch 33/100
816/816 [==============================] - 1s - loss: 3.3723e-04 - rmse: 0.0182 - val_loss: 7.8868e-04 - val_rmse: 0.0275
Epoch 34/100
816/816 [==============================] - 1s - loss: 3.0839e-04 - rmse: 0.0174 - val_loss: 6.8244e-04 - val_rmse: 0.0256
Epoch 35/100
816/816 [==============================] - 1s - loss: 4.1118e-04 - rmse: 0.0200 - val_loss: 6.2974e-04 - val_rmse: 0.0248
Epoch 36/100
816/816 [==============================] - 1s - loss: 3.4059e-04 - rmse: 0.0183 - val_loss: 6.1464e-04 - val_rmse: 0.0244
Epoch 37/100
816/816 [==============================] - 1s - loss: 3.1219e-04 - rmse: 0.0175 - val_loss: 6.7945e-04 - val_rmse: 0.0256
Epoch 38/100
816/816 [==============================] - 1s - loss: 3.1646e-04 - rmse: 0.0176 - val_loss: 7.6362e-04 - val_rmse: 0.0270
Epoch 39/100
816/816 [==============================] - 1s - loss: 3

816/816 [==============================] - 2s - loss: 1.2863e-04 - rmse: 0.0113 - val_loss: 8.0267e-04 - val_rmse: 0.0277
Epoch 93/100
816/816 [==============================] - 2s - loss: 1.1295e-04 - rmse: 0.0106 - val_loss: 9.9031e-04 - val_rmse: 0.0307
Epoch 94/100
816/816 [==============================] - 2s - loss: 1.2051e-04 - rmse: 0.0109 - val_loss: 7.7116e-04 - val_rmse: 0.0271
Epoch 95/100
816/816 [==============================] - 2s - loss: 1.1454e-04 - rmse: 0.0107 - val_loss: 7.2535e-04 - val_rmse: 0.0263
Epoch 96/100
816/816 [==============================] - 2s - loss: 1.0281e-04 - rmse: 0.0101 - val_loss: 8.7608e-04 - val_rmse: 0.0288
Epoch 97/100
816/816 [==============================] - 2s - loss: 1.0166e-04 - rmse: 0.0100 - val_loss: 8.5400e-04 - val_rmse: 0.0285
Epoch 98/100
816/816 [==============================] - 2s - loss: 1.0918e-04 - rmse: 0.0104 - val_loss: 8.9870e-04 - val_rmse: 0.0293
Epoch 99/100
816/816 [==============================] - 2s - loss: 1